In [1]:
import os
import xml.etree.ElementTree as ET

def convert(size, box):
    # No normalization, just return the coordinates as they are
    x_center = (box[0] + box[1]) / 2.0
    y_center = (box[2] + box[3]) / 2.0
    width = box[1] - box[0]
    height = box[3] - box[2]
    return (x_center, y_center, width, height)

def convert_annotation(xml_file, output_dir):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    
    output_file = os.path.join(output_dir, os.path.basename(xml_file).replace('.xml', '.txt'))
    with open(output_file, 'w') as out_file:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls not in class_mapping:
                continue
            cls_id = class_mapping[cls]
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
                 float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((w, h), b)
            out_file.write(f"{cls_id} {' '.join([str(a) for a in bb])}\n")

# Path to your XML files and the directory where you want to save the YOLO-format annotations
xml_dir = 'C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/pascal voc 1.1/Annotations'
output_dir = 'C:/Users/Anirudh/OneDrive/Desktop/out'
os.makedirs(output_dir, exist_ok=True)

# Class mapping (adjust as needed)
class_mapping = {
    'odometer': 0
}

# Convert all XML files in the directory
for xml_file in os.listdir(xml_dir):
    if xml_file.endswith('.xml'):
        convert_annotation(os.path.join(xml_dir, xml_file), output_dir)

print("Conversion completed!")


Conversion completed!


In [9]:
import os
import cv2

def draw_annotations(image_path, annotation_path, output_dir):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to open image file {image_path}")
        return

    with open(annotation_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            
            # Convert center coordinates to top-left and bottom-right coordinates
            xmin = int(x_center - width / 2)
            ymin = int(y_center - height / 2)
            xmax = int(x_center + width / 2)
            ymax = int(y_center + height / 2)
            
            # Draw the bounding box
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            # Put class id text on the bounding box
            cv2.putText(image, str(class_id), (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, image)

# Paths to the directories containing images and annotations
image_dir = 'C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images'
annotation_dir = 'C:/Users/Anirudh/OneDrive/Desktop/output'
output_dir = 'C:/Users/Anirudh/OneDrive/Desktop/annotated_images/'
os.makedirs(output_dir, exist_ok=True)

# Process each image and its corresponding annotation file
for image_file in os.listdir(image_dir):
    if image_file.endswith('.jpg'):
        image_path = os.path.join(image_dir, image_file)
        annotation_path = os.path.join(annotation_dir, image_file.replace('.jpg', '.txt'))
        
        if os.path.exists(annotation_path):
            draw_annotations(image_path, annotation_path, output_dir)
        else:
            print(f"Warning: Annotation file for {image_file} not found.")

print("Annotated images have been saved.")


KeyboardInterrupt: 

In [24]:
import os
import shutil
import random

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def split_dataset(images_dir, annotations_dir, output_dir, train_ratio=0.8):
    # Create output directories
    train_images_dir = os.path.join(output_dir, 'train', 'images')
    train_labels_dir = os.path.join(output_dir, 'train', 'labels')
    val_images_dir = os.path.join(output_dir, 'val', 'images')
    val_labels_dir = os.path.join(output_dir, 'val', 'labels')

    create_dir(train_images_dir)
    create_dir(train_labels_dir)
    create_dir(val_images_dir)
    create_dir(val_labels_dir)

    # List all images and shuffle them
    images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
    random.shuffle(images)

    # Calculate split index
    split_idx = int(len(images) * train_ratio)

    # Split images into training and validation sets
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    # Copy training images and annotations
    for image in train_images:
        base_name = os.path.splitext(image)[0]
        annotation = base_name + '.txt'

        shutil.copy(os.path.join(images_dir, image), os.path.join(train_images_dir, image))
        shutil.copy(os.path.join(annotations_dir, annotation), os.path.join(train_labels_dir, annotation))

    # Copy validation images and annotations
    for image in val_images:
        base_name = os.path.splitext(image)[0]
        annotation = base_name + '.txt'

        shutil.copy(os.path.join(images_dir, image), os.path.join(val_images_dir, image))
        shutil.copy(os.path.join(annotations_dir, annotation), os.path.join(val_labels_dir, annotation))

    print("Dataset split completed.")

# Define paths
images_dir = 'C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images'
annotations_dir = 'C:/Users/Anirudh/OneDrive/Desktop/normalized'
output_dir = 'C:/Users/Anirudh/OneDrive/Desktop/Dataset'

# Split the dataset
split_dataset(images_dir, annotations_dir, output_dir, train_ratio=0.8)


Dataset split completed.


New https://pypi.org/project/ultralytics/8.2.25 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.19  Python-3.10.1 torch-2.3.0+cpu CPU (Intel Core(TM) i7-10750H 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:/Users/Anirudh/OneDrive/Desktop/Odometer OCR.v2i.yolov8/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, ret

train: Scanning C:\Users\Anirudh\OneDrive\Desktop\Odometer OCR.v2i.yolov8\train\labels.cache... 1638 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1638/1638 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 14, len(boxes) = 22820. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning C:\Users\Anirudh\OneDrive\Desktop\Odometer OCR.v2i.yolov8\valid\labels.cache... 467 images, 0 backgrounds, 0 corrupt: 100%|██████████| 467/467 [00:00<?, ?it/s]


Plotting labels to runs\detect\train8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000588, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train8
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/103 [00:00<?, ?it/s]

: 

In [2]:
import os
import cv2
import xml.etree.ElementTree as ET

def resize_image_and_annotations(image_dir, annotation_dir, output_dir, target_size):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Loop through all image files in the image directory
    for filename in os.listdir(image_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Check if file is an image
            image_path = os.path.join(image_dir, filename)
            annotation_file = os.path.join(annotation_dir, filename.replace('.jpg', '.xml').replace('.jpeg', '.xml').replace('.png', '.xml'))
            
            # Load image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Unable to read image '{filename}'")
                continue
            
            # Resize image
            resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
            # Write resized image
            resized_image_path = os.path.join(output_dir, filename)
            cv2.imwrite(resized_image_path, resized_image)
            
            # Resize annotations
            resize_annotations(annotation_file, target_size, output_dir)

def resize_annotations(annotation_file, target_size, output_dir):
    # Parse XML annotation file
    tree = ET.parse(annotation_file)
    root = tree.getroot()
    size = root.find('size')
    orig_width = int(size.find('width').text)
    orig_height = int(size.find('height').text)
    
    # Calculate resize ratio
    resize_ratio_w = target_size[0] / orig_width
    resize_ratio_h = target_size[1] / orig_height
    
    # Loop through all object annotations
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        
        # Resize bounding box coordinates
        xmin_resized = int(xmin * resize_ratio_w)
        ymin_resized = int(ymin * resize_ratio_h)
        xmax_resized = int(xmax * resize_ratio_w)
        ymax_resized = int(ymax * resize_ratio_h)
        
        # Update bounding box coordinates in the XML annotation
        bbox.find('xmin').text = str(xmin_resized)
        bbox.find('ymin').text = str(ymin_resized)
        bbox.find('xmax').text = str(xmax_resized)
        bbox.find('ymax').text = str(ymax_resized)
    
    # Write the updated XML annotation to a new file
    resized_annotation_file = os.path.join(output_dir, os.path.basename(annotation_file))
    tree.write(resized_annotation_file)

# Define paths and parameters
image_dir = 'C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images'
annotation_dir = 'C:/Users/Anirudh/OneDrive/Desktop/out'
output_dir = '"C:/Users/Anirudh/OneDrive/Desktop/anno"'
target_size = (640, 640)

# Resize images and annotations
resize_image_and_annotations(image_dir, annotation_dir, output_dir, target_size)

print("Resizing completed!")


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: '"C:'

In [20]:
import os
import cv2

def resize_image_and_annotations(image_dir, annotation_dir, output_dir, target_size):
    # Create output directory if not exists
    os.makedirs(output_dir, exist_ok=True)

    # Resize images and update annotations
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            # Read image
            image_path = image_dir+'/'+filename
            print(image_path)
            image = cv2.imread(image_path)

            # Resize image
            resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)

            # Write resized image
            resized_image_path = os.path.join(output_dir, filename)
            cv2.imwrite(resized_image_path, resized_image)

            # Update corresponding annotation
            annotation_file = os.path.join(annotation_dir, os.path.splitext(filename)[0] + '.txt')
            update_annotation(annotation_file, image.shape, resized_image.shape, target_size)

def update_annotation(annotation_file, original_image_size, resized_image_size, target_size):
    # Read annotation file
    with open(annotation_file, 'r') as f:
        lines = f.readlines()

    # Update bounding box coordinates
    resize_factor_x = target_size[0] / original_image_size[1]
    resize_factor_y = target_size[1] / original_image_size[0]

    updated_lines = []
    for line in lines:
        parts = line.strip().split(' ')
        class_id = parts[0]
        x_center = float(parts[1]) * resize_factor_x
        y_center = float(parts[2]) * resize_factor_y
        width = float(parts[3]) * resize_factor_x
        height = float(parts[4]) * resize_factor_y

        # Convert back to YOLO format
        updated_line = f"{class_id} {x_center} {y_center} {width} {height}\n"
        updated_lines.append(updated_line)

    # Write updated annotation file
    output_annotation_file = os.path.join(output_dir, os.path.basename(annotation_file))
    with open(output_annotation_file, 'w') as f:
        f.writelines(updated_lines)

# Paths to your image and annotation directories
image_dir = 'C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images'
annotation_dir = 'C:/Users/Anirudh/OneDrive/Desktop/output'
output_dir = 'C:/Users/Anirudh/OneDrive/Desktop/resized'

# Target size for resizing
target_size = (640, 640)

# Resize images and annotations
resize_image_and_annotations(image_dir, annotation_dir, output_dir, target_size)


C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images/00000002-PHOTO-2020-11-20-11-21-22.jpg
C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images/00000003-PHOTO-2020-11-20-11-21-23.jpg
C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images/00000004-PHOTO-2020-11-20-11-21-25.jpg
C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images/00000005-PHOTO-2020-11-20-11-21-26.jpg
C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images/00000006-PHOTO-2020-11-20-11-21-26.jpg
C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images/00000007-PHOTO-2020-11-20-11-21-27.jpg
C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images/00000008-PHOTO-2020-11-20-11-21-28.jpg
C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images/00000009-PHOTO-2020-11-20-11-21-29.jpg
C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images/00000010-PHOTO-2020-11-20-11-21-30.jpg
C:/Users/Anirudh/On

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [22]:
import os
import xml.etree.ElementTree as ET

def convert_annotation(xml_file, output_dir):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    output_file = os.path.join(output_dir, os.path.basename(xml_file).replace('.xml', '.txt'))
    with open(output_file, 'w') as out_file:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls == 'odometer':  # Skip 'odometer' class
                continue
            xmlbox = obj.find('bndbox')
            # Get bounding box coordinates
            xmin = float(xmlbox.find('xmin').text)
            ymin = float(xmlbox.find('ymin').text)
            xmax = float(xmlbox.find('xmax').text)
            ymax = float(xmlbox.find('ymax').text)
            # Convert to YOLO format
            x_center = (xmin + xmax) / (2 * w)
            y_center = (ymin + ymax) / (2 * h)
            width = (xmax - xmin) / w
            height = (ymax - ymin) / h
            # Write to YOLO format file
            out_file.write(f"{cls} {x_center} {y_center} {width} {height}\n")

# Path to the directory containing XML annotations
xml_dir = 'C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images'
# Output directory for YOLO format text files
output_dir = 'C:/Users/Anirudh/OneDrive/Desktop/normalized'

# Convert all XML files in the directory
for xml_file in os.listdir(xml_dir):
    if xml_file.endswith('.xml'):
        convert_annotation(os.path.join(xml_dir, xml_file), output_dir)

print("Conversion completed!")


Conversion completed!


In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET

def resize_image_and_annotations(image_dir, annotation_dir, output_dir, target_size):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Loop through all image files in the image directory
    for filename in os.listdir(image_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Check if file is an image
            image_path = os.path.join(image_dir, filename)
            annotation_file = os.path.join(annotation_dir, filename.replace('.jpg', '.xml').replace('.jpeg', '.xml').replace('.png', '.xml'))
            
            # Load image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Unable to read image '{filename}'")
                continue
            
            # Resize image
            resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
            # Write resized image
            resized_image_path = os.path.join(output_dir, filename)
            cv2.imwrite(resized_image_path, resized_image)
            
            # Resize annotations
            resize_annotations(annotation_file, target_size, output_dir)

def resize_annotations(annotation_file, target_size, output_dir):
    # Parse XML annotation file
    tree = ET.parse(annotation_file)
    root = tree.getroot()
    size = root.find('size')
    orig_width = int(size.find('width').text)
    orig_height = int(size.find('height').text)
    
    # Calculate resize ratio
    resize_ratio_w = target_size[0] / orig_width
    resize_ratio_h = target_size[1] / orig_height
    
    # Loop through all object annotations
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        
        # Resize bounding box coordinates
        xmin_resized = int(xmin * resize_ratio_w)
        ymin_resized = int(ymin * resize_ratio_h)
        xmax_resized = int(xmax * resize_ratio_w)
        ymax_resized = int(ymax * resize_ratio_h)
        
        # Update bounding box coordinates in the XML annotation
        bbox.find('xmin').text = str(xmin_resized)
        bbox.find('ymin').text = str(ymin_resized)
        bbox.find('xmax').text = str(xmax_resized)
        bbox.find('ymax').text = str(ymax_resized)
    
    # Write the updated XML annotation to a new file
    resized_annotation_file = os.path.join(output_dir, os.path.basename(annotation_file))
    tree.write(resized_annotation_file)

# Define paths and parameters
image_dir = 'C:/Users/Anirudh/OneDrive/Desktop/mahindra/trodo-v01/trodo-v01/images'
annotation_dir = 'path/to/annotations'
output_dir = 'path/to/output'
target_size = (640, 640)

# Resize images and annotations
resize_image_and_annotations(image_dir, annotation_dir, output_dir, target_size)

print("Resizing completed!")


In [1]:
from ultralytics import YOLO

# Load a YOLOv8 model
model = YOLO('yolov8s.pt')  # You can use 'yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x' based on your requirement

# Train the model
model.train(data='C:/Users/Anirudh/OneDrive/Desktop/output/data.yaml', epochs=25, imgsz=640)

New https://pypi.org/project/ultralytics/8.2.26 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.19  Python-3.10.1 torch-2.3.0+cpu CPU (Intel Core(TM) i7-10750H 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:/Users/Anirudh/OneDrive/Desktop/output/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False,

train: Scanning C:\Users\Anirudh\OneDrive\Desktop\output\train\labels... 3763 images, 2 backgrounds, 0 corrupt: 100%|██████████| 3763/3763 [00:06<00:00, 585.98it/s]


train: New cache created: C:\Users\Anirudh\OneDrive\Desktop\output\train\labels.cache
WARNING  Box and segment counts should be equal, but got len(segments) = 1406, len(boxes) = 4438. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


val: Scanning C:\Users\Anirudh\OneDrive\Desktop\output\val\labels... 1075 images, 2 backgrounds, 0 corrupt: 100%|██████████| 1075/1075 [00:01<00:00, 584.82it/s]


val: New cache created: C:\Users\Anirudh\OneDrive\Desktop\output\val\labels.cache
WARNING  Box and segment counts should be equal, but got len(segments) = 410, len(boxes) = 1271. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
Plotting labels to runs\detect\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train10
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/236 [00:00<?, ?it/s]

: 

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# load image from the IAM dataset
url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [3]:
import transformers
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests

# load image from the IAM database (actually this model is meant to be used on printed text)
url = 'https://fki.tic.heia-fr.ch/static/img/a01-122-02-00.jpg'
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-printed')
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


ModuleNotFoundError: No module named 'transformers'

: 